In [52]:
#THIS BLOCK IS FOR IMPORTING AND READING GENERAL INFO ABOUT DATA
#Library managment is made through anaconda environments

import mne
import numpy as np
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

from scipy.signal import butter, iirnotch, sosfilt, zpk2sos
from scipy import signal

#Upload arquivo EDF
path =r'C:\Users\thaismarchetti\Documents\Doutorado\Dados\Experimentos\paired_pulse_context_tree\Preliminary_tests\CR_05_27_2025\emg_data_tree.edf'
raw = mne.io.read_raw_edf(path, preload=True)

#Check if the data was loaded and give general info about it
print(raw.info)
print(raw.ch_names)

#PLOT THE RAW DATA CORRESPONDING TO THE EMG CHANNELS (Don't forget to change channel name)
raw.plot(picks=['Input 33'], scalings='auto', title='Raw EEG Data', show=True)

sampling_rate = int(raw.info['sfreq'])
print(sampling_rate)

Extracting EDF parameters from C:\Users\thaismarchetti\Documents\Doutorado\Dados\Experimentos\paired_pulse_context_tree\Preliminary_tests\CR_05_27_2025\emg_data_tree.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 12924999  =      0.000 ...  2585.000 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: Input 33
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 2500.0 Hz
 meas_date: 2025-05-27 14:01:45 UTC
 nchan: 1
 projs: []
 sfreq: 5000.0 Hz
 subject_info: 5 items (dict)
>
['Input 33']
5000


In [53]:
# THIS BLOCK APPLIES FILTERS TO THE DATA

# Apply a high-pass filter at 20 Hz  using an IIR Butterworth filter
filt_raw = raw.copy().filter(l_freq=20, h_freq=None, picks=['Input 33'], method="iir", n_jobs=2, iir_params=dict(order=8, ftype="butter"))

# Load the filtered data
filt_raw.load_data()

# Define the frequencies for the notch filter to remove powerline noise and harmonics
freqs = (60, 120, 180, 240, 300, 360)

# Apply the notch filter 
filt_raw.notch_filter(freqs=freqs, trans_bandwidth=2, notch_widths=2, picks=[0])
data, times =filt_raw[:,:] ##### Não entendi essa linha
# Extract data for the first channel

emg_data = filt_raw.get_data()
print(emg_data.shape)
filt_raw.plot(picks=['Input 33'], scalings='auto', title='Raw EEG Data', show=True)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 20 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoff at 20.00 Hz: -6.02 dB



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 1.00 Hz
- Upper transition bandwidth: 1.00 Hz
- Filter length: 16501 samples (3.300 s)



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


(1, 12925000)


<MNEBrowseFigure size 1920x1009 with 4 Axes>

In [54]:
# THIS BLOCK GETS THE ONSETS (TRIGGER INSTANTS) AND SEPARETES THEM IN A AND B

onsets = raw.annotations.onset
#print(raw.annotations.onset)

descriptions = raw.annotations.description
trigger_a_onsets = [onsets[i] for i in range(len(descriptions)) if "A" in descriptions[i]]
trigger_b_onsets = [onsets[i] for i in range(len(descriptions)) if "B" in descriptions[i]]

trigger_a_onsets = [float(onset) for onset in trigger_a_onsets]
trigger_b_onsets = [float(onset) for onset in trigger_b_onsets]

print("Onsets Trigger A:", trigger_a_onsets)
print(len(trigger_a_onsets))
print("Onsets Trigger B:", trigger_b_onsets)

tempo=trigger_a_onsets
len(trigger_a_onsets) 

emg_data = filt_raw.copy().pick('Input 33').get_data()
print("EMG data:", emg_data)
print(emg_data.shape)

Onsets Trigger A: [27.1192, 34.7192, 43.0378, 51.9114, 60.5194, 69.8258, 78.2644, 86.836, 94.7272, 102.3216, 109.9708, 119.2276, 127.0038, 135.8116, 143.3564, 152.2434, 159.9394, 168.953, 176.6338, 185.54, 193.0984, 201.3588, 209.2276, 216.8578, 225.0576, 234.2528, 243.5492, 251.1426, 258.673, 267.855, 277.2914, 285.2106, 293.528, 302.5864, 311.4702, 319.2258, 327.1292, 335.5872, 344.9264, 352.4474, 361.8656, 369.6152, 378.8776, 388.0172, 396.2298, 405.407, 414.909, 423.4526, 432.7376, 440.3614, 449.0862, 457.1264, 466.5856, 475.5924, 483.8494, 493.2676, 502.1336, 510.1212, 518.794, 528.2478, 536.3008, 545.212, 554.0494, 563.0248, 572.2096, 579.7466, 588.8152, 596.7798, 605.469, 614.9328, 622.7122, 630.822, 638.7488, 647.0168, 654.5854, 663.059, 672.1644, 680.2108, 688.7488, 696.4638, 705.3122, 713.4172, 722.6664, 730.993, 739.3778, 748.3694, 757.2054, 766.3088, 774.9212, 782.8474, 790.965, 798.8632, 807.4938, 816.3538, 825.666, 833.7628, 841.3738, 849.9928, 857.832, 865.8338, 874.1606

In [48]:
#THIS BLOCK SEARCHES FOR THE MAXIMUM AND MINIMUN IN A TIME INTERVAL (GIVEN THE ONSETS)

artefact_escape_ms = 13 / 1000
valor_ms = 60
valor_x = valor_ms / 1000
fs=5000
maximo_emg = []
minimo_emg = []
filtered_signal= emg_data.T

for t in tempo:
    # Calcular os índices com base no tempo
    indice_trigger = int((t + artefact_escape_ms) * fs)
    print(indice_trigger)
    indice_final_busca = int((t + valor_x) * fs)
    print(indice_final_busca)
    
   
    maximo_emg.append(np.max(filtered_signal[indice_trigger:indice_final_busca]))
    minimo_emg.append(np.min(filtered_signal[indice_trigger:indice_final_busca]))
    
    # Create a scatter plot for the start of the searching
    plt.scatter(indice_trigger, np.zeros_like(indice_trigger), color='k', marker='*', label='Trigger')

    # Create a scatter plot for the end of the searching interval
    #OBS: THE ASTERISK THAT DELIMITS THE SEARCH INTERVAL ARE VERY SMALL
    plt.scatter(indice_final_busca, np.zeros_like(indice_final_busca), color='red', marker='*', label='Final Busca')

indices = np.array(raw.annotations.onset) * fs

plt.plot(filtered_signal)

for index in indices:
    plt.axvline(x=index, color='green', linestyle='--', label='Event')
   
plt.show()
   
# Converter para arrays numpy se necessário
maximo_emg = np.array(maximo_emg)
minimo_emg = np.array(minimo_emg)

print("Máximos EMG:", maximo_emg)
print("Mínimos EMG:", minimo_emg)

135661
135896
173661
173896
215253
215489
259622
259857
302662
302897
349194
349429
391387
391622
434245
434480
473701
473936
511673
511908
549919
550154
596203
596438
635084
635319
679123
679358
716847
717082
761282
761517
799762
799997
844830
845065
883234
883469
927765
928000
965557
965792
1006859
1007094
1046203
1046438
1084354
1084589
1125353
1125588
1171329
1171564
1217811
1218046
1255778
1256013
1293430
1293665
1339340
1339575
1386522
1386757
1426117
1426353
1467705
1467940
1512997
1513232
1557415
1557651
1596193
1596429
1635711
1635946
1678000
1678236
1724697
1724932
1762302
1762537
1809392
1809627
1848141
1848376
1894452
1894687
1940151
1940386
1981214
1981449
2027099
2027335
2074609
2074845
2117328
2117563
2163753
2163988
2201872
2202107
2245496
2245731
2285697
2285932
2332993
2333228
2378027
2378262
2419312
2419547
2466403
2466638
2510733
2510968
2550671
2550906
2594035
2594269
2641304
2641538
2681569
2681803
2726125
2726359
2770312
2770546
2815189
2815424
2861113
2861348
28

In [13]:
# THIS BLOCK CONVERTS THE EMG VALUES TO MICRO VOLTS AND SAVES THE DATA
MEP=(maximo_emg-minimo_emg)*1000000
MEP
np.savetxt('MEP_microvoltsvalues_CR.txt',MEP, fmt='%.5f')